In [1]:
import json
import numpy as np
import gurobipy
import utils
from gurobipy import Model, GRB, quicksum
import pandas as pd
import networkx as nx
import plotly.express as px

from utils import create_planning, create_tables, dominate, ModelData, ModelPreordreData, get_preorder, create_graph

In [2]:
m, data = utils.create_model('instances/toy_instance.json')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-12


In [3]:
def get_non_dominated(model: Model, data: ModelData):
    solutions = []
    m.params.outputflag = 0
    m.NumScenarios = (data.Nj+1)*(data.Nm*data.Np + 1)

    c1 = m.addConstr(data.f2 == 0)
    c2 = m.addConstr(data.f3 == 0)

    for i in range(data.Nj+1):
        for j in range(data.Nm*data.Np+1):
            m.params.ScenarioNumber = i*(data.Nm*data.Np+1) + j
            m.ScenNName = 'i = {}, j = {}'.format(i, j)
            c1.ScenNRhs = j
            c2.ScenNRhs = i

    print("Starting optimzation...")

    m.setObjective(data.f1)
    m.reset()
    m.optimize()

    print("done.")

    for s in range(m.NumScenarios):
        m.params.ScenarioNumber = s
        solutions.append([m.ScenNObjVal, data.Af.ScenNX.sum(), data.Dm.ScenNX])

    m.remove(c1)
    m.remove(c2)

    solutions = np.array(solutions)
    solutions = solutions.round().astype(int)

    filtered_solutions = []
    scenario = []

    for i in range(solutions.shape[0]):
        non_dominated = True
        for j in range(solutions.shape[0]):
            if i == j:
                continue
            if dominate(solutions[j], solutions[i]):
                non_dominated = False        
                break
        if non_dominated:
            scenario.append(i)
            filtered_solutions.append(solutions[i])

    return filtered_solutions, scenario 
        

In [4]:
filtered_solutions, scenario = get_non_dominated(m, data)

Starting optimzation...
done.


In [5]:
df_solutions = pd.DataFrame(-1*np.array(filtered_solutions))
df_solutions['scenario'] = scenario
df_solutions.rename(columns={0: 'f1', 1: 'f2', 2:'f3'}, inplace=True)

In [6]:
preference_list = [
    (7,   9),
    (10, 15),
    (7,  1)
]

choices = filtered_solutions.copy()
preordre_model, preordre_data = get_preorder(choices, preference_list)

G = create_graph(preordre_data)
df_solutions['degree'] = [val for idx, val in G.degree]
df_solutions.sort_values(by='degree', ascending=False, inplace=True)

df_solutions

,f1,f2,f3,scenario,degree
7,59,-11,-1,27,20
6,49,-9,-1,25,19
5,47,-8,-1,24,17
3,30,-5,-1,21,15
4,37,-6,-1,22,14
12,50,-5,-2,37,14
17,42,-3,-3,51,14
11,40,-4,-2,36,13
10,30,-3,-2,35,13
16,32,-2,-3,50,12


In [7]:
fig = px.scatter_3d(df_solutions, x='f1', y='f2', z='f3', color='degree')
fig.update_layout(
    autosize=False,
    width=1000,
    height=400,
    )

fig.update_traces(marker={'size': 5})
fig.show()

In [8]:
m.params.ScenarioNumber = df_solutions.iloc[0]['scenario']
tab_a, tab_b = create_tables(data)
display(tab_a)
display(tab_b)
create_planning(data)

,A,B,C
membre,,,
Olivia,1,1,1
Liam,1,1,0
Emma,0,0,1


,A,B,C
projet,,,
Job1,1,1,1
Job2,1,2,0
Job3,1,0,2
Job4,0,2,1
Job5,0,0,2


,0,1,2,3,4
membre,,,,,
Olivia,C_Job5,X,A_Job1,C_Job3,B_Job4
Liam,X,X,X,X,B_Job4
Emma,C_Job5,X,C_Job1,C_Job3,C_Job4
